<font style= "font-size: 50px">
    
                \    |    /
                 \   |   /                 
                  \  |  /                     
                   \ | /
           --------- • ---------
                   / | \
                  /  |  \
                 /   |   \
                /    |    \

</font>

###### 

# <center> <i> Data Aggregation &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</i> </center>

# 

### Welcome:
Without talking the bu\**it we will start it right away.

# 

In [1]:
import pandas as pd
import numpy as np

Here we will talk about the ways of aggregation.

Simply put... there are a range of aggfuncs, and some of them are listed:
* mean, median, sum, min, max
* last, first
* prod

The `last, first` looks different. Let's take a look at them.

# 

In [2]:
df = pd.DataFrame(np.random.randint(0, 100, (10, 2)), columns= ['a', 'b'])

In [3]:
df.insert(0, 'key', ['A', 'A', 'B', 'B', 'C', 'A', 'C', 'C', 'C', 'B'])

In [4]:
df

,key,a,b
0,A,65,67
1,A,11,29
2,B,29,58
3,B,19,0
4,C,15,72
5,A,0,21
6,C,74,58
7,C,74,93
8,C,43,57
9,B,51,2


# 

## → `.groupby().first() / .last()` 

These first and last are different. The `DF.first() / .last()` are about the TIME SERIES. Here, is the `.groupby().first() / .last()` which is...

Well, is [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.first.html)

# 

____

<br>According to the book:
> *Internally, GroupBy efficiently slices up the Series and after applying the addfunc, it combines them in to the result object*

____

Looking it's internal function...

In [5]:
counter = 0
def own_func(ser):
    global counter
    counter += 1
    print("( ", counter, " )")
    print(ser, end= "\n" * 3)

In [6]:
df.groupby('key').agg(own_func)

(  1  )
0    65
1    11
5     0
Name: a, dtype: int32


(  2  )
2    29
3    19
9    51
Name: a, dtype: int32


(  3  )
4    15
6    74
7    74
8    43
Name: a, dtype: int32


(  4  )
0    67
1    29
5    21
Name: b, dtype: int32


(  5  )
2    58
3     0
9     2
Name: b, dtype: int32


(  6  )
4    72
6    58
7    93
8    57
Name: b, dtype: int32




,a,b
key,,
A,None,None
B,None,None
C,None,None


It goes from column to column... as there are three groups and two columns... it takes a column and then for each group it passes a series.

So, for that we can do whatever on the series and return the **value** ***or*** **a series**.

# 

## → Trying to **`use OWN function`** in groupby *(first time)*

In [7]:
df

,key,a,b
0,A,65,67
1,A,11,29
2,B,29,58
3,B,19,0
4,C,15,72
5,A,0,21
6,C,74,58
7,C,74,93
8,C,43,57
9,B,51,2


Remember: This is NOT an APPLY function. It is the groupby. So the things would work accordingly.

In [8]:
def own_func(ser):
    return ser.max() - ser.min()

In [9]:
df.groupby('key').agg(own_func)

,a,b
key,,
A,65,46
B,32,58
C,59,36


**Interpretation:** It will put a difference from the that group of that column. So here for example of A group, we are getting the 47. Which means that for all A's in the column 'a' the difference between min and max is 47. 

Now, we can retuen more arguments too!

In [13]:
def own_func(ser):
    return pd.Series([ser.min(), ser.max()], index= ['a', 'b'])

In [14]:
df.groupby('key').agg(own_func)

,a,b
key,,
A,key A a 0 b 21 dtype: object,key A a 65 b 67 dtype: object
B,key B a 19 b 0 dtype: object,key B a 51 b 58 dtype: object
C,key C a 15 b 57 dtype: object,key C a 74 b 93 dtype: object


Here, a column is having min and b is having max.<br>
Now quite useful... but still it is it.

# 

##  → Naming in multiple aggs

In [18]:
df

,key,a,b
0,A,65,67
1,A,11,29
2,B,29,58
3,B,19,0
4,C,15,72
5,A,0,21
6,C,74,58
7,C,74,93
8,C,43,57
9,B,51,2


In [21]:
def own_func(ser):
    return ser.min()

In [22]:
df.groupby('key').agg(['mean', lambda x: x.max(), own_func])

a                        b                    
          mean <lambda_0> own_func mean <lambda_0> own_func
key                                                        
A    25.333333         65        0   39         67       21
B    33.000000         51       19   20         58        0
C    51.500000         74       15   70         93       57

Now, see that - **EVERY function** has a `__name__` attribute while lambda doesn't. So it returns the column names as the`__name__` of the function. 

What if you wanna get more descriptive? **THIS WAS THE QUESTION THAT I WANTED TO BE SOLVED**

#### USE `[ (name1, func1), (name2, func2) ]`.
In other words, a list of tuple of name and function!

# 

In [23]:
df.groupby('key').agg([('average', 'mean'),
                       ('min', lambda x: x.max()),
                       ('max', own_func)])

a               b        
       average min max average min max
key                                   
A    25.333333  65   0      39  67  21
B    33.000000  51  19      20  58   0
C    51.500000  74  15      70  93  57

## BRO!! Look at that ↑↑

# 

# → Avoiding the group categories to be an INDEX.

In [24]:
df.groupby("key").mean()

,a,b
key,,
A,25.333333,39.0
B,33.000000,20.0
C,51.500000,70.0


See ↑ A, B, C are the indices... to avoid that you can either use `reset_index` or just use the built-in parameter - **`as_index = False`**

In [25]:
df.groupby("key", as_index= False).mean()

,key,a,b
0,A,25.333333,39.0
1,B,33.000000,20.0
2,C,51.500000,70.0


# 

## → Now, difference between `.agg()` and `.apply()` with groupby.

See... 

In [47]:
def own_func(ser):
    global counter
    counter += 1
    print("( ", counter, " )")
    
    print(ser, f"type: {type(ser)}", end= "\n" * 3, sep= "\n")
    

In [48]:
# Using agg
counter = 0
df.groupby('key').agg(own_func)

(  1  )
0    65
1    11
5     0
Name: a, dtype: int32
type: <class 'pandas.core.series.Series'>


(  2  )
2    29
3    19
9    51
Name: a, dtype: int32
type: <class 'pandas.core.series.Series'>


(  3  )
4    15
6    74
7    74
8    43
Name: a, dtype: int32
type: <class 'pandas.core.series.Series'>


(  4  )
0    67
1    29
5    21
Name: b, dtype: int32
type: <class 'pandas.core.series.Series'>


(  5  )
2    58
3     0
9     2
Name: b, dtype: int32
type: <class 'pandas.core.series.Series'>


(  6  )
4    72
6    58
7    93
8    57
Name: b, dtype: int32
type: <class 'pandas.core.series.Series'>




,a,b
key,,
A,None,None
B,None,None
C,None,None


In [49]:
# Using apply
counter = 0
df.groupby('key').apply(own_func)

(  1  )
  key   a   b
0   A  65  67
1   A  11  29
5   A   0  21
type: <class 'pandas.core.frame.DataFrame'>


(  2  )
  key   a   b
2   B  29  58
3   B  19   0
9   B  51   2
type: <class 'pandas.core.frame.DataFrame'>


(  3  )
  key   a   b
4   C  15  72
6   C  74  58
7   C  74  93
8   C  43  57
type: <class 'pandas.core.frame.DataFrame'>




""


See? the difference is, **apply is more general** which works with WHOLE group, while **agg is more specific** which applies to per column and per group. 

### So, 
When you want to get - let's say top 5 rows per group - you wanna use the `apply` but if you want to get more techy, then you might want to touch `agg`... 

Both of them are simple, and self explanatory. You may want to observe how both of them work. And how do they return the values.

# 

# That's it
Next up, we will talk about some ... I don't know ... but some unknown things.<br>
Stay curious ∞